In [1]:
import os

In [2]:
%pwd

'd:\\MLOPS\\ML-Approach-for-Predict-Cancellation-Prevent-Loss-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'd:\\MLOPS\\ML-Approach-for-Predict-Cancellation-Prevent-Loss-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir : Path
    unzip_data_dir : Path
    STATUS_FILE : str 
    all_schema : dict  

In [6]:
from src.constants import *
from src.utils.common import *

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir= config.root_dir,
            unzip_data_dir= config.unzip_data_dir,
            STATUS_FILE= config.STATUS_FILE,
            all_schema= schema
        )

        return data_validation_config
        

In [8]:
import os
import sys
import pandas as pd
from src.utils.logger import logging
from src.utils.exception import CustomException

In [9]:
class DataValidation:
    def __init__(self, config=DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status : {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status : {validation_status}")

            return validation_status
          
        except Exception as e:
            raise CustomException(e,sys)

In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()

except Exception as e:
    raise CustomException(e,sys)

[ 2024-04-01 19:52:27,505 ] 31 root - INFO - config\config.yaml yaml file loaded successfully
[ 2024-04-01 19:52:27,520 ] 31 root - INFO - schema.yaml yaml file loaded successfully
[ 2024-04-01 19:52:27,529 ] 52 root - INFO - Created directory at : artifacts
[ 2024-04-01 19:52:27,532 ] 52 root - INFO - Created directory at : artifacts/data_validation
